## To use this notebook

Jupyter Notebooks allow you to modify and run the code in this document. To run a section (known as a 'cell',) select it and then use CTRL + ENTER, or select the play button on the toolbar above. Note that each section already has some example output beneath it, so you can see what the results of running a cell will look like.

NOTE: You must run each cell in order, from top to bottom. Running cells out of order can result in an error.

## Requirements

* An Azure Virtual Network
* A Spark on HDInsight 3.6 cluster, inside the virtual network
* A Kafka on HDInsight cluster, inside the virtual network

## Load packages

To use Spark structured streaming with Kafka, you must load the spark-sql-kafka package. The version must match the version of both kafka and Spark that you are using. The name of the package contains the versions that it works with. For example, `spark-sql-kafka-0-10_2.11:2.1.0` works with the following versions:

* Kafka 0.10
* Spark 2.1.0
* Scala 2.11

Run the next cell to load a package that works with Kafka on HDInsight 3.6, and Spark 2.1 on HDInsight 3.6.

In [1]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.1.0", 
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1497883131614_0004,spark,idle,Link,Link,


## Define a schema for the data
When reading data from Kafka, the data is provided in the 'value' column. In this example, the data is a JSON document that describes a Tweet. Run the following cell to create a schema for the JSON document structure.

In [2]:
// Import bits useed for declaring schemas and working with JSON data
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

// Define the structure of the Twitter JSON document that is read from Kafka
// Note, this isn't pretty, but there's some odd behavior where moving .add to 
// a new line causes an error.
val schema = (new StructType).add("created_at", StringType).add("id", LongType).add("id_str", StringType).add("text", StringType).add("source", StringType).add("truncated", BooleanType).add("in_reply_to_status_id", LongType).add("in_reply_to_status_id_str", StringType).add("in_reply_to_user_id", LongType).add("in_reply_to_user_id_str", StringType).add("in_reply_to_screen_name", StringType).add("user", (new StructType).add("id", LongType)
        .add("id_str", StringType)
        .add("name", StringType)
        .add("screen_name", StringType)
        .add("location", StringType)
        .add("url", StringType)
        .add("description", StringType)
        .add("protected", BooleanType)
        .add("verified", BooleanType)
        .add("followers_count", LongType)
        .add("friends_count", LongType)
        .add("listed_count", LongType)
        .add("favourites_count", LongType)
        .add("statuses_count", LongType)
        .add("created_at", StringType)
        .add("utc_offset", IntegerType)
        .add("time_zone", StringType)
        .add("geo_enabled", BooleanType)
        .add("lang", StringType)
        .add("contributors_enabled", BooleanType)
        .add("is_translator", BooleanType)
        .add("profile_background_color", StringType)
        .add("profile_background_image_url", StringType)
        .add("profile_background_image_url_https", StringType)
        .add("profile_background_tile", BooleanType)
        .add("profile_link_color", StringType)
        .add("profile_sidebar_border_color", StringType)
        .add("profile_sidebar_fill_color", StringType)
        .add("profile_text_color", StringType)
        .add("profile_use_background_image", BooleanType)
        .add("profile_image_url", StringType)
        .add("profile_image_url_https", StringType)
        .add("profile_banner_url", StringType)
        .add("default_profile", BooleanType)
        .add("default_profile_image", BooleanType)
        .add("following", StringType)
        .add("follow_request_sent", StringType)
        .add("notifications", StringType)).add("geo", StringType).add("coordinates", StringType).add("place", StringType).add("contributors", StringType).add("is_quote_status", BooleanType).add("retweet_count", LongType).add("favorite_count", LongType).add("entities", (new StructType)
        .add("hashtags", ArrayType((new StructType)
            .add("text", StringType)
            .add("indices", ArrayType(LongType)))).add("urls", ArrayType((new StructType)
            .add("url", StringType)
            .add("expanded_url", StringType)
            .add("display_url", StringType)
            .add("indices", ArrayType(LongType))))
        .add("user_mentions", ArrayType(StringType))
        .add("symbols", ArrayType(StringType))).add("favorited", BooleanType).add("retweeted", BooleanType).add("possibly_sensitive", BooleanType).add("filter_level", StringType).add("lang", StringType).add("timestamp_ms", StringType)

// Display a tree view of the schema.
schema.printTreeString

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1497883131614_0005,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
root
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- id_str: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- truncated: boolean (nullable = true)
 |-- in_reply_to_status_id: long (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |-- in_reply_to_user_id: long (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- id_str: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- protected: boolean (nullable = true)
 |    |-- verified: boolean (nullable = true)
 |    |-- followers_count: long (n

## Read the data and apply the schema

In the following cell, set `kafkaBrokers` to the broker hosts for your Kafka cluster. The value should be a comma-delimited list of the hosts, similar to the following example:

    wn0-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092,wn1-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092,wn2-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092
        
To find the Kafka brokers information for your Kafka on HDInsight cluster, you can use the Ambari REST API. The following examples demonstrate how to retrieve this information using the the `curl` and `jq` utilities (from Bash) or Windows PowerShell:

* From __Bash__ or other Unix shell:

    ```bash
CLUSTERNAME='the name of your HDInsight cluster'
PASSWORD='the password for your cluster login account'
curl -u admin:$PASSWORD -G "https://$CLUSTERNAME.azurehdinsight.net/api/v1/clusters/$CLUSTERNAME/services/KAFKA/components/KAFKA_BROKER" | jq -r '["\(.host_components[].HostRoles.host_name):9092"] | join(",")'
    ```

    * From __Azure Powershell__:

    ```powershell
$creds = Get-Credential -UserName "admin" -Message "Enter the HDInsight login"
$clusterName = Read-Host -Prompt "Enter the Kafka cluster name"
$resp = Invoke-WebRequest -Uri "https://$clusterName.azurehdinsight.net/api/v1/clusters/$clusterName/services/KAFKA/components/KAFKA_BROKER" `
    -Credential $creds
$respObj = ConvertFrom-Json $resp.Content
$brokerHosts = $respObj.host_components.HostRoles.host_name
($brokerHosts -join ":9092,") + ":9092"
    ```

When you run the next cell, it starts processing data from Kafka from the beginning of the topic. The results are written to the console (displayed as cell output) once the stream ends.

The output of the stream should appear similar to the following text:

```
+------------------+---------------+--------------------+
|                id|           name|                text|
+------------------+---------------+--------------------+
|876795956712353792|Alexey Kuritsyn|RT @SnoopyG7: One...|
|876795954329989120|       amber ☀︎|Stained my shirt ...|
|876795955538006016|        Omalley|RT @kentkristense...|
|876795954229374976|     オタク文化は日本文化|「お、ARKがsteamで980円...|
|876795957559443457|  Simon Carless|Oo, Nex Machina (...|
|876795956687130624| Common Grounds|Coffee - it's wha...|
|876795953264697344|    Scott Crisp|RT @saiyanisland:...|
+------------------+---------------+--------------------+
```

In [3]:
// kafkaBrokers should contain a comma-delimited list of brokers. For example:
// kafkaBrokers = "wn0-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092,wn1-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092,wn2-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092"
val kafkaBrokers="wn0-kafka.mtyhrma3s4du5ba5ezzeido3se.dx.internal.cloudapp.net:9092,wn1-kafka.mtyhrma3s4du5ba5ezzeido3se.dx.internal.cloudapp.net:9092,wn2-kafka.mtyhrma3s4du5ba5ezzeido3se.dx.internal.cloudapp.net:9092"

// Read from the Kafka stream source
val kafka = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafkaBrokers).option("subscribe", "tweets").option("startingOffsets","earliest").load()

/* Select the following columns from the Kafka data:
   * value - the JSON data for a tweet
   Use from_json to apply the schema and store the schematized data in the 'tweet' column
*/
val tweetData=kafka.select(
    from_json(col("value").cast("string"), schema) as "tweet")

// There's a lot of data in the Twitter JSON object. Just grab the tweet ID, user name, and text
val tweetText=tweetData.select("tweet.id",
                               "tweet.user.name",
                               "tweet.text")

// Start writing the stream to the console. Use a timeout so that control is returned to the notebook.
tweetText.writeStream.format("console").start.awaitTermination(30000)

-------------------------------------------
Batch: 0
-------------------------------------------
+------------------+-----------------+--------------------+
|                id|             name|                text|
+------------------+-----------------+--------------------+
|876837578472919042|    Cathal Cusack|RT @Xbox: Get a f...|
|876837586823782403|       jenny oran|@BasicallyIDoWrk ...|
|876837587897503744|Lady Angela アンジェラ|RT @NintendoSwitc...|
|876837582306512896|  gawana twioloni|WWE All Stars (Mi...|
|876837583497703424|           REDD..|RT @LikwidCyance:...|
|876837589939912704|              ゴン太|gpdwinでようやくツクール作品...|
|876837591009468416|         K.Sunaga|もうPS4買っちゃおうかな
2回払...|
|876837591278080001|         RedSnake|RT @saiyanisland:...|
|876837590409891842|     bee 🔮| nsfr|RT @chrizjoseph: ...|
|876837592423059457|             Will|RT @Minumuss: The...|
|876837594109267968|   Gretchen Estep|RT @tay123bay: Ca...|
|876837594016792576|              Tim|Today's breakfast...|
|876